In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [2]:
env = gym.make("FrozenLake-v1")

In [3]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [4]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [5]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.001
exploration_decay_rate = 0.001


In [6]:
rewards_all_episodes = []
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        state = new_state
        rewards_current_episode += reward  
        if done == True: 
            break 
        # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

    rewards_all_episodes.append(rewards_current_episode) 

In [7]:
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.04900000000000004
2000 :  0.21800000000000017
3000 :  0.43900000000000033
4000 :  0.6080000000000004
5000 :  0.6750000000000005
6000 :  0.7040000000000005
7000 :  0.7280000000000005
8000 :  0.7480000000000006
9000 :  0.7350000000000005
10000 :  0.6840000000000005


In [8]:
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.47041245 0.45909328 0.4613016  0.46085705]
 [0.32098836 0.40823287 0.32760355 0.45715325]
 [0.40992971 0.40260103 0.39967218 0.43274613]
 [0.24210916 0.3842819  0.18111957 0.42359627]
 [0.47737741 0.36409981 0.37804606 0.32675126]
 [0.         0.         0.         0.        ]
 [0.15508934 0.15671503 0.29236821 0.07734351]
 [0.         0.         0.         0.        ]
 [0.39825222 0.35359545 0.43274832 0.49347752]
 [0.45145982 0.49798806 0.47989279 0.4024277 ]
 [0.42884097 0.37022331 0.34355939 0.26142849]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.51524892 0.54895321 0.63064552 0.39233488]
 [0.71026537 0.71945072 0.70960216 0.71072011]
 [0.         0.         0.         0.        ]]


In [10]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break          

        # Set new state
        state = new_state

env.close()

****You reached the goal!****
